## Векторная модель текста в NLTK
### Задание 1
Возьмем файл alice.txt с прошлого семинара. Мы уже научились разбивать его на токены и удалять стоп-слова.
Используя nltk.FreqDist, выведите 20 самых часто встречающихся слов в тексте (не удаляя стоп символы).

Ответьте на вопросы:

    Какое слово стоит на 1 месте?
    Сколько раз встречается слово, стоящее на 20 месте?
    Сколько не стоп-слов среди первых десяти?
    Сколько раз встречается слово "крокет"? (понятно, что оно не входит в top20)


In [ ]:
# d1 = nltk.FreqDist(l1)
# d1.most_common(10)

# d3['...'], d3.freq('...')

### Задание 2
Избавьтесь от стоп-слов с помощью списка stopwords из nltk.corpus. Используйте pymorphy2 или Mystem, чтобы привести слова к нормальной форме. Ответьте на вопросы:

    Какое слово стоит на 1 месте?
    Сколько раз встречается слово, стоящее на 20 месте?
    Остались ли какие-то стоп-слова среди первых 10? 20?
    Сколько раз встречается слово "крокет"? Почему результат изменился?


In [ ]:
# morph = pymorphy2.MorphAnalyzer()
# l3 = [morph.parse(token)[0].normal_form for token in l1 if not token in stopwords.words('russian')]

# d3['...'], d3.freq('...')

Давайте пострим кумулятивный график частот слов. Как он выглядит и почему так? Попробуйте поменять количество слов, какие тенденции вы видите?


In [ ]:
d3.plot (50, cumulative = True)

### Задание 3

Найдите распределение длин слов в тексте, выведите top5 и ответьте на вопросы:

    Сколько всего в данном тексте вариантов длин слов (вместе со стоп-словами)?
    Слов какой длины в тексте больше всего?
    Какую долю занимают слова этой длины?
    Сколько слов длины 1?

Выведите в алфавитном поредке длинные слова (длиннее 15 символов).

    Сколько их?

In [ ]:
# fdist = nltk.FreqDist(len(w) for w in l1)  
# print(fdist)  
# print (fdist.most_common(5))
# print (fdist.freq(fdist.max()))

# long_words = [w for w in l1 if len(w) > 10]
# sorted(long_words)

### Задание 4

Превратите строку в NLTK-текст и проверьте, как работают спецфункции, на собственных примерах.


In [ ]:
# text = nltk.Text(l1)
# print (text.concordance(".....")) 
# print (text.similar('......'))
# print (text.common_contexts([".....", "......"]))
# print (text.collocations(30))

## Эмбеддинги


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
from sklearn.metrics import *
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')
random.seed(1228)

%matplotlib inline

Скачиваем куски датасета твитов ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/ilkte35m35l38mr/negative.sql).

In [ ]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

In [ ]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

Загружаем лемматизированные статьи без стоп-слов и создаем массив текстов

In [ ]:
from pymystem3 import Mystem
import re


m = Mystem()


regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""



def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [ ]:
df_neg = pd.read_csv("negative.csv?dl=0", sep=';', header = None, usecols = [3])
df_pos = pd.read_csv("positive.csv?dl=0", sep=';', header = None, usecols = [3])
df_neg['sent'] = 'neg'
df_pos['sent'] = 'pos'
df_pos['text'] = df_pos[3]
df_neg['text'] = df_neg[3]
df = pd.concat([df_neg, df_pos])
df = df[['text', 'sent']]
%time df.text = df.text.apply(words_only)
%time df.text = df.text.apply(lemmatize)

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.tail()

In [ ]:
texts = [df.text.iloc[i].split() for i in range(len(df))]

### Обучение модели в gensim

In [ ]:
texts[0]

In [ ]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(texts, size=100, window=5, min_count=5, workers=4)
model.save("word2v.model")

In [ ]:
# здесь можно посмотреть все слова, к которым мы имеем вектора
# model.vocab

Загружаем обученную модель (для скорости):

In [ ]:
# from gensim.models import Word2Vec
# model = Word2Vec.load("sent_w2v.model")

Теперь можно воспользjваться функциями поиска похожих / непохожих слов на данное

    model.most_similar
    model.doesnt_match
    etc

In [ ]:
model.most_similar("корпоратив")

In [ ]:
model.most_similar(positive=["король","добрый"], negative=["хороший"])

In [ ]:
model.doesnt_match("борщ сметана макароны пирожок котлета".split())

### Задание 5
Выделите top 100 самых частотных слов и получите для них только что обученные вектора

In [ ]:
from nltk import FreqDist

fd = FreqDist()
# your code is hear

top_words = # your code is hear

In [ ]:
model['рабочий']

In [ ]:
top_words_vec = # your code is hear

Визуализируем вектора. Для этого воспользуемся методом снижения размерности t-sne

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Теперь нарисуем облако слов

In [ ]:
from wordcloud import *
word_freq = [i for i in fd.most_common(100)]
wd = WordCloud(background_color = 'white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure()
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()

### Кластеризация слов

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(top_words_vec)

# Языковые модели

Какое слово в последовательности вероятнее: 

Поезд прибыл на
* вокзал
* север

Какая последовательность вероятнее:
* Вокзал прибыл поезд на
* Поезд прибыл на вокзал

Языковая модель [language model, LM]  позволяет оценить вероятность следующего слова в последовательности  $P(w_n | w_1, \ldots, w_{n-1})$ и оценить вероятность всей последовательности слов $P(w_1, \ldots, w_n)$.

### Приложения:
* Задачи, в которых нужно обработать сложный и зашумленный вход: распознавание речи, распознавание сканированных и рукописных текстов;
* Исправление опечаток
* Машинный перевод
* Подсказка при наборе 

### Виды моделей:
* Счетные модели
    * цепи Маркова
* Нейронные модели, обычно реккурентные нейронные сети с LSTM/GRU
* seq2seq архитектуры

## Модель $n$-грам

Пусть $w_{1:n}=w_1,\ldots,w_m$ – последовательность слов.

Цепное правило:
$ P(w_{1:m}) = P(w_1) P(w_2 | w_1) P(w_3 | w_{1:2}) \ldots P(w_m | w_{1:m-1}) = \prod_{k=1}^{m} P(w_k | w_{1:k-1}) $

Но оценить $P(w_k | w_{1:k-1})$ не легче! 

Переходим к $n$-грамам: $P(w_{i+1} | w_{1:i}) \approx P(w_{i+1} | w_{i-n:i})  $ , то есть, учитываем $n-1$ предыдущее слово. 

Модель
* униграм: $P(w_k)$
* биграм: $P(w_k | w_{k-1})$
* триграм: $P(w_k | w_{k-1} w_{k-2})$


Т.е. используем Марковские допущения о длине запоминаемой цепочки.

* Вероятность следующего слова в последовательности: $ P(w_{i+1} | w_{1:i}) \approx P(w_{i-n:i}) $
* Вероятность всей последовательности слов: $P(w_{1:n}) = \prod_{k=1}^l P(w_k | w_{k-n+1: k-1}) $

### Качество модели  $n$-грам

Перплексия: насколько хорошо модель предсказывает выборку. Чем ниже значение перплексии, тем лучше.

$PP(\texttt{LM}) = 2 ^ {-\frac{1}{m} \log_2 \texttt{LM} (w_i | w_{1:i-1})}$

## Счетные языковые модели

### ММП оценки вероятностей
$ P_{MLE}(w_k | w_{k-n+1:k-1}) = \frac{\texttt{count}(w_{k-n+1:k-1} w_k )}{\texttt{count}(w_{k-n+1:k-1} )} $

В модели биграм:

$ P_{MLE}(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k )}{\texttt{count}(w_{k-1} )} $

Возникает проблема нулевых вероятностей!

### Аддитивное сглаживание Лапласа

$ P(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k ) + \alpha}{\texttt{count}(w_{k-1} ) + \alpha |V|} $

![AiB](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/aib.png)  

BOS А и Б сидели на трубе EOS

BOS А упало Б пропало EOS

BOS что осталось на трубе EOS




$P($ и $| $ A $) = \frac{1}{2}$

$P($ Б $| $ и $) = \frac{1}{1}$

$P($ трубе $| $ на $) = \frac{2}{2}$

$P($ сидели $| $ Б $) = \frac{1}{2}$

$P($ на $| $ сидели $) = \frac{1}{2}$


## Модели биграм в NLTK

In [ ]:
import nltk

from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout

In [ ]:
names = [name.strip().lower() for name in open('dinos.txt').readlines()]
print(names[:10])

In [ ]:
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)

print(list(freq.keys()))

In [ ]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))
cfreq['a']

In [ ]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

In [ ]:
from math import log
log(cprob['a'].prob('a')) + log(cprob['a'].prob('b')) + log(cprob['a'].prob('c'))

In [ ]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l
print('p(a) = %1.4f' %unigram_prob('a'))

In [ ]:
[bi for bi in nltk.bigrams('aachenosaurus')]

#### Задание 1

1. Напишите функцию для оценки вероятности имени динозавра. 
2. Найдите наиболее вероятное имя динозавра из данного списка. 

In [ ]:
cprob["a"].generate()

#### Задание 2

Напишите функцию для генерации нового имени динозавра фиксированной длины. (если еще не сделали это на прошлом занятии :) )

## Нейронные языковые модели

* Вход: $n$-грамы $w_{1:k}$
* $v(w_i)$ – эмбеддинг слова $w_i$, $v(w_i) \in \mathbb{R}^{d_{emb}}$, $d_{emb}$ – размерность эмбеддинга, $v(w) = E_{[w]}$
* $x = [v(w_1), v(w_2), \ldots , v(w_k)]$

$\widehat{y} = P(w_i | w_{1:k} ) = \texttt{LM}(w_{1:k}) = \texttt{softmax}(hW^2 +b^2)$

$h = g(xW^1+b^1)$

$w_i \in V$, $E \in \mathbb{R}^{|V|\times d_{emb}}, W^1 \in \mathbb{R}^{k \cdot d_{emb} \times d_{hid}}, b^1 \in \mathbb{R} ^ {d_{hid}}, W^2 \in \mathbb{R}^{d_{hid} \times |V|}, b^2 \in \mathbb{R} ^ {|V|}$

![nnlm](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/nlm1.png)

### Семплирование в нейронных языковых моделях 
### (Генерация текстов с помощью нейронных языковых моделей)

1. Задать начальную последовательность символов длины $k$ (/слов)
2. Предсказать распределение вероятностей слов с условием на $k$ предыдущих слов
3. 1. Выбрать слово с наибольшей вероятностью
3. 2. Выбрать слово по предсказаному распределению
4. Сдвинуть окно на одно слово и повторить 

#### Линейный поиск  (beam search)
Всегда помним $h$ наиболее вероятных гипотез:
1. Для генерации первого слова в последоватительности генерируем $h$ кандидатов, а не 1
2. Генерируем $h \times h$ кандидатов для второго слова и храним только $h$ наиболее вероятных


![nnlm](https://raw.githubusercontent.com/artemovae/nlp-course-sberbank/e70ab4acb696c00e170ea91d1bb28fd9ba31c170/img/nlm2.png)

In [ ]:
alphabet = list(set(chars))
print('total chars:', len(alphabet))

In [ ]:
maxlen = 5
step = 1
ngrams = []
next_chars = []
for name in names:
    for i in range(0, len(name) - maxlen, step):
        ngrams.append(' '.join([char for char in name[i: i + maxlen]]))
        next_chars.append(name[i + maxlen])
print('nb ngrams:', len(ngrams))
print(ngrams[0],next_chars[0])
print(ngrams[1],next_chars[1])

In [ ]:
tokenizer = Tokenizer(num_words=len(alphabet))
tokenizer.fit_on_texts(ngrams)

sequences = tokenizer.texts_to_sequences(ngrams)
X_train = pad_sequences(sequences, maxlen=maxlen)
sequences = tokenizer.texts_to_sequences(next_chars)
y_train = tokenizer.sequences_to_matrix(sequences)
X_train[0]

In [ ]:
y_train[0]

In [ ]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [ ]:
model = Sequential()

model.add(Embedding(len(alphabet), 50, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation = 'softmax'))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'softmax'))
model.add(Dropout(0.2))
model.add(Dense(len(alphabet), activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train,y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose = 0)

In [ ]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) #/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)), p = preds)
    return probas

In [ ]:
import random
import sys
import numpy as np

generated = ''
seed = 'katya'
generated += seed
print('----- Generating with seed: "' + seed + '"')
print(generated)

for i in range(8):
    sequences = tokenizer.texts_to_sequences([' '.join([char for char in generated[-maxlen:]])])
    X_pred = pad_sequences(sequences, maxlen=maxlen)
    preds = model.predict(X_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = index_char[next_index]
    generated += next_char
    print(generated)

#### Задание 3

Измените код выше так, чтобы генерировались панграмы – имена динозавров, не содержащие повторяющихся букв

#### Задание 4

Измените функцию семлирования `sample`: добавьте параметр `t`, изпольузуемый для шкалирования вероятностей  `preds`: ```
preds /= t
``` 

Как использование этого параметра влияет на генерируемые имена?

In [ ]:
from scipy.cluster.hierarchy import  ward, dendrogram

linkage_matrix = ward(dist) 

fig, ax = plt.subplots(figsize=(10, 100)) 
ax = dendrogram(linkage_matrix, orientation="right", labels=top_words);

plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')

plt.tight_layout() 

plt.savefig('w2v_cluster.png', dpi=200) #save figure as ward_clusters